In [1]:
import utils
import pickle
import os
import pandas as pd
import visualisation
import hueristics


os.environ['DIR'] = "/home/ml/softezza_ml/"

models_dir = '/home/ml/softezza_ml/models/knn'
models_meta = pd.read_csv('models/knn/meta.csv').drop(columns='Unnamed: 0')

models_meta

,Recall@10,MAP@10,MIUF@10,Siren@10,PopInt@10,model,K,B,K1
0,0.104393,0.038852,5.294475,0.000210,0.099451,bm25,50.0,0.25,2.50
1,0.105175,0.039246,5.273610,0.000209,0.100029,bm25,50.0,0.50,0.50
2,0.110683,0.040791,5.185683,0.000203,0.101525,bm25,100.0,0.25,2.50
3,0.112202,0.041445,5.158078,0.000202,0.104386,bm25,100.0,0.50,0.50
4,0.116439,0.042601,5.134638,0.000199,0.110904,bm25,200.0,0.25,2.50
5,0.117926,0.043239,5.106162,0.000198,0.113707,bm25,200.0,0.50,0.50
6,0.119831,0.045402,4.931020,0.000179,0.131334,bm25,50.0,0.25,1.25
7,0.125847,0.047755,4.824942,0.000171,0.138100,bm25,100.0,0.25,1.25
8,0.131121,0.049233,4.790733,0.000167,0.147111,bm25,200.0,0.25,1.25


In [2]:
config = utils.DataConfig(
    experiment=utils.Experiment.LIGHT_FM,
    split_strategy=utils.TimeSortSplit('all', 0.6, 0.2, 0.2),
    filter_strategy=[
        utils.MinNumInteractionsFilter(10, 500),
        utils.OnlyLastInteractionsFilter('user_id', 20)
    ],
    concat_stages=True,
    use_popular_penalty=False
)

data = utils.load_data(config)
data.train_interactions.head()

Data after filter:
Len of train interactions with period [['2023-08-24T00:28:10.000000000'] / ['2023-08-24T00:28:09.000000000']] - 6002853
Len of test interactions with period [['2023-09-14T06:04:34.000000000'] / ['2023-10-04T11:31:55.000000000']] - 509348
Num of uniq users 364765Num of uniq items 8223


,user_id,item_id,timestamp,weight,index
0,15847001,tt0317219,2023-08-24 00:28:09,0.929345,3384613
1,29835903,tt0947798,2023-08-24 00:28:07,0.952778,3384614
2,24085201,tt5761544,2023-08-24 00:28:04,0.950840,3384615
3,68446701,tt0099472,2023-08-24 00:28:04,0.990000,3384616
4,19975701,tt13932162,2023-08-24 00:28:04,0.939482,3384617


In [3]:
test_users = utils.get_users_for_test(data.train_interactions, min_n_interactions=3, max_n_interactions=5)

train_dataset, test_dataset = data.get_rectools_dataset()
params2model_data = {}

for p in models_meta[['model', 'K', 'K1', 'B']].itertuples(False, None):
    
    with open(os.path.join(models_dir, f"{p[0]}_K{p[1]}_K1{p[2]}_B{p[3]}.pickle"), mode='rb') as f:
        params2model_data[p] = {
            'model': pickle.load(f)
        }
    
    params2model_data[p]['reco'] = params2model_data[p]['model'].recommend(
        k=10,
        users=test_users.users_idx,
        dataset=train_dataset,
        filter_viewed=True,
        add_rank_col=True,
    )

list(params2model_data.items())[0]

(('bm25', 50.0, 2.5, 0.25),
 {'model': <rectools.models.implicit_knn.ImplicitItemKNNWrapperModel at 0x7fbd06ec3b50>,
  'reco':      user_id     item_id        score  rank
  0   55809401   tt9100018  3835.390368     1
  1   55809401  tt13405778  2892.865501     2
  2   55809401  tt11456054  2858.286529     3
  3   55809401   tt7631568  1924.563676     4
  4   55809401  tt11703244  1881.426022     5
  ..       ...         ...          ...   ...
  95  34062801   tt1853739  1339.423510     6
  96  34062801   tt8688634  1247.469450     7
  97  34062801   tt0368933  1019.788551     8
  98  34062801  tt14349032  1009.474286     9
  99  34062801   tt2584384   981.534826    10
  
  [100 rows x 4 columns]})

In [4]:
item_id2meta = pd.read_csv('/home/ml/softezza_ml/static_mappers/item_id2meta.csv').set_index('item_id', drop=True)
item_id2title = pd.read_csv('/home/ml/softezza_ml/static_mappers/item_id2title.csv').set_index('item_id', drop=True)['title']


item_id2meta

,type,year,rank,runtime,genres,company,director,writer,cast,mppa,num_views
item_id,,,,,,,,,,,
tt0004972,1,1915,6.2,195,"""Drama / History / War""","""David W. Griffith Corp., Epoch Producing Corp...","""D.W. Griffith""","""Thomas Dixon Jr., Thomas Dixon Jr., Thomas Di...","""Lillian Gish, Mae Marsh, Henry B. Walthall""","""TV-PG""",2580
tt0006864,1,1916,7.7,163,"""Drama / History""","""D.W. Griffith Productions""","""D.W. Griffith""","""Hettie Grey Baker, Tod Browning, D.W. Griffith""","""Lillian Gish, Mae Marsh, Robert Harron""","""N""",70
tt0010323,1,1920,8.0,76,"""Horror / Mystery / Thriller""","""Decla-Bioscop AG""","""Robert Wiene""","""Carl Mayer, Hans Janowitz""","""Werner Krauss, Conrad Veidt, Friedrich Feher""","""TV-PG""",551
tt0011237,1,1920,7.2,76,"""Fantasy / Horror""","""Projektions-AG Union (PAGU)""","""Paul Wegener, Carl Boese""","""Paul Wegener, Henrik Galeen""","""Paul Wegener, Albert Steinrück, Lyda Salmonova""","""N""",57
tt0011841,1,1920,7.3,145,"""Drama / Romance""","""D.W. Griffith Productions""","""D.W. Griffith""","""Lottie Blair Parker, William A. Brady, Joseph...","""Lillian Gish, Richard Barthelmess, Mrs. David...","""N""",40
...,...,...,...,...,...,...,...,...,...,...,...
tt9894470,1,2019,6.1,92,"""Action / Crime / Horror""","""Fangoria, Channel 83 Films, Media Finance Cap...","""Joe Begos""","""Max Brallier, Matthew McArdle""","""Stephen Lang, William Sadler, Fred Williamson""","""N""",2109
tt9900092,2,2020,7.3,42,"""Drama / Fantasy / Sci-Fi / Thriller""","""""","""""","""Eliot Laurence, Eliot Laurence, Eliot Laurence""","""Taylor Hickson, Amalia Holm, Demetria McKinney""","""TV-14""",0
tt9902160,1,2020,7.0,97,"""Drama""","""BBC Films, British Film Institute (BFI), Elem...","""Phyllida Lloyd""","""Clare Dunne, Malcolm Campbell, Clare Dunne""","""Molly McCann, Clare Dunne, Ruby Rose O'Hara""","""R""",5112


In [5]:
named_recos = []

for params, model_data in params2model_data.items():
    r = model_data['reco']
    r['model_name'] = f"{params[0]} [K: {params[1]}, K1: {params[2]}, B: {params[3]}]"
    named_recos.append(r)


test_interactions = []

for user_id, history in test_users.users_histories.items():
    hist_data = pd.DataFrame({'item_id': history})
    hist_data['user_id'] = user_id
    test_interactions.append(hist_data)


vis_interactions = pd.concat(test_interactions)
vis_recos = pd.concat(named_recos)
vis_dict = dict(zip([f"user_{i}" for i in range(1, 11)], test_users.users_idx))

vis_items_data = pd.DataFrame({'item_id': data.all_items})
vis_items_data['title'] = vis_items_data['item_id'].apply(lambda iid: item_id2title.loc[iid])
vis_items_data['watched_in_all_time'] = vis_items_data['item_id'].apply(lambda iid: item_id2meta.loc[iid, 'num_views'])
vis_items_data['release_year'] = vis_items_data['item_id'].apply(lambda iid: item_id2meta.loc[iid, 'year'])
vis_items_data['genres'] = vis_items_data['item_id'].apply(lambda iid: item_id2meta.loc[iid, 'genres'])


visualisation.Showcase(
    interactions=vis_interactions,
    full_recos=vis_recos,
    users_dict=vis_dict,
    item_data=vis_items_data,
    convert_ids_to_int=False
);